In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#Just to push


In [ ]:
img = cv2.imread('rov_pool.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
edges = cv2.Canny(gray, 106, 107, apertureSize=3) # detect edges, gray is image in grayscale, 50 and 150 represent 2 images that have been threshholded at 2 different levels, apertureSize controls how much light the image gets and how exposed it is
lines = cv2.HoughLinesP(
                edges, #described above
                1, #1 pixel resolution parameter
                np.pi/180, # 1 degree resolution parameter
                60, #min number of intersections/votes
                minLineLength=475,
                maxLineGap=100,
        ) # detect lines

for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    slope = (y2-y1)/(x2-x1)
    print(str(slope))

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
def drawLines(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 90, 100, apertureSize=3) 
    lines = cv2.HoughLinesP(
                    edges, #described above
                    1, #1 pixel resolution parameter
                    np.pi/180, # 1 degree resolution parameter
                    10, #min number of intersections/votes
                    minLineLength=10,
                    maxLineGap=60,
            ) # detect lines
    try:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            slope = (y2-y1)/(x2-x1)
            print(str(slope))
    except TypeError:
        pass
    
    return img


In [ ]:
import cv2
import numpy as np

video = cv2.VideoCapture('AUV_Vid.mkv')

fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_file = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file, fourcc, 30, (width, height))

ret, frame = video.read()
count = 0
frequency = 1

while ret:
    if count % frequency == 0:
        processed_frame = drawLines(frame)
        output_video.write(processed_frame)

    count += 1
    ret, frame = video.read()

video.release()
output_video.release()

In [ ]:
video = cv2.VideoCapture('AUV_Vid.mkv')
ret, frame = video.read()
count = 0
frequency = 150
while ret:

    if count % frequency == 0:
        plt.imshow(cv2.cvtColor(drawLines(frame), cv2.COLOR_BGR2RGB))
        plt.show()
        
    count+=1
    ret, frame = video.read()


In [ ]:
from dt_apriltags import Detector
img = cv2.imread('test_image.png', cv2.IMREAD_GRAYSCALE)

at_detector = Detector(families='tag36h11', #controls what it is supposed to detect
                       nthreads=1, #controls the number of threads used in the detection
                       quad_decimate=1.0, #controls to what scale the image is lowered in resolution
                       quad_sigma=0.0, #blur applied to the image to reduce noise
                       refine_edges=1, #attempts to redefine the edges of the tag boundaries
                       decode_sharpening=0.25, #controls how much the image is sharpened after the tags are discovered
                       debug=0) # no debugging information will be produced

tags = at_detector.detect(img, estimate_tag_pose=False, camera_params=None, tag_size=None)

color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

for tag in tags:
    for idx in range(len(tag.corners)):
        cv2.line(color_img, tuple(tag.corners[idx - 1, :].astype(int)), tuple(tag.corners[idx, :].astype(int)), (0, 255, 0))

    cv2.putText(color_img, str(tag.tag_id),
                org=(tag.corners[0, 0].astype(int) + 10, tag.corners[0, 1].astype(int) + 10),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.8,
                color=(0, 0, 255))
    
plt.imshow(img)

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dt_apriltags import Detector
import matplotlib.cm as cm
from lane_detection import *
from lane_following import *

video = cv2.VideoCapture('AUV_Vid.mkv')

width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_file = 'output_video_center.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file, fourcc, 30, (width, height))

i = 0
ret = True
while ret:
    slopeSet = set() 
    ret, frame = video.read()
    if(i%50 == 0 and i > 90):
        lines = detect_lines(frame, 50, 90, 3,150,10)
        #print (lines)
    
        #try: 
        #    frame = draw_lines(frame, lines,(0, 255, 0))
        #    print ("tried to draw lines")
        #except :
       #     pass
        try:
            lanes = detect_lanes(lines)
            #print ("tried to detect lanes")
            pickedLane = pick_lane(lanes)
            frame = draw_Single_lane(frame, pickedLane, (255, 0, 0))
            frame = draw_lines(frame, lines,(0, 255, 0))
            # center_intercept, center_slope = get_lane_center(pickedLane)
            # xPoint = pickedLane[0][2]
            # yPoint = pickedLane[0][3]
            # cv2.line(frame, (int(center_intercept), 1080), (int(xPoint), int(yPoint)), (0,0,255), 3)
        except:
            pass
        
    
        output_video.write(frame)



video.release()
output_video.release()

slopeList:[3.742857142857143, -9.533333333333333, 3.7131147540983607, 7.266666666666667, 4.3090909090909095, 4.288888888888889, -9.6875, 7.275862068965517, 9.75, 4.739130434782608, 9.772727272727273]
xInterceptList:[1355.1832061068703, 613.1608391608391, 1359.399558498896, 1098.7614678899083, 1287.8649789029537, 1295.979274611399, 614.4516129032259, 1104.8104265402844, 1037.871794871795, 1237.5412844036698, 1031.613953488372]
slopeList:[-2.606299212598425, 6.466666666666667, -2.6031746031746033]
xInterceptList:[173.57401812688818, 1079.5567010309278, 172.77439024390247]
slopeList:[-1.8048780487804879, -1.8095238095238095]
xInterceptList:[187.24324324324323, 188.31578947368416]
slopeList:[-11.458333333333334, -4.029411764705882, -30.2, 3.3055555555555554, -28.0, -2.9074074074074074, 4.022222222222222, 61.0, 4.04054054054054, -7.083333333333333, -3.253731343283582, 59.333333333333336, -5.130434782608695, -2.9074074074074074, 3.297872340425532, 5.239130434782608, -4.0588235294117645, -5.2

/home/travistran/cv-intro/lane_detection.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  intercept = ((((2160 - y1)/slope)  )+ x1)
/home/travistran/cv-intro/lane_detection.py:105: RuntimeWarning: divide by zero encountered in scalar divide
  slopeDiff = abs(1/ slopeList[i]-1 /slopeList[j])


slopeList:[0.0, -1.1126760563380282, -2.4827586206896552, 0.05113636363636364]
xInterceptList:[inf, -694.5316455696202, 139.48611111111114, 33791.77777777778]
slopeList:[0.08426966292134831, 0.08609271523178808, -0.22839506172839505, -2.8793103448275863, -0.24870466321243523]
xInterceptList:[20632.066666666666, 20226.69230769231, -5964.918918918919, -172.14371257485027, -5419.166666666667]
slopeList:[0.12162162162162163, 0.12658227848101267]
xInterceptList:[13537.555555555555, 13058.0]
slopeList:[-0.30670926517571884, -0.3041958041958042, -0.2656826568265683, -0.3037974683544304, -0.2706422018348624, -0.3037383177570093, -0.2677595628415301, -0.3105590062111801, 0.12169312169312169, -0.3063583815028902]
xInterceptList:[-3502.229166666667, -3536.6321839080456, -4382.5, -3537.5, -4282.050847457627, -3543.492307692308, -4328.510204081633, -3458.5400000000004, 13096.782608695652, -3509.5094339622638]
slopeList:[0.2111111111111111, 0.21171171171171171, 0.2119205298013245, -0.446366782006920

/home/travistran/cv-intro/lane_detection.py:109: RuntimeWarning: divide by zero encountered in scalar divide
  xPoint = ((slopeList[i] * xInterceptList[i]) - (slopeList[j] * xInterceptList[j]))/(slopeList[i]-slopeList[j])
